# What is `xarray` and how to use for simple applications

Xarray is a Python library that simplifies working with multi-dimensional labeled arrays. It is especially useful for working with netCDF files and other scientific datasets, such as those used in climate and meteorological research. With Xarray, you can easily perform indexing, slicing, aggregations, and group-by operations on multi-dimensional data, making data analysis, visualization, and processing more intuitive and efficient.

You can also uxe `xarray` directly from R. Instead of using `import` do this:

``` r
library(reticulate)
xr <- import("xarray")
```

Then, you can call the functions as:

``` r
ds <- xr$open_dataset("dataset path...")
```

And methods with:

``` r
ds$to_netcdf()
```

That is, just change the `.` (dot) by `$` (dollar)

I will show two examples. The first, using a `netCDF` file. The other, accessing directly the ERDDAP server. You can also open `zarr`, `tif` and other formats. For a very nice example, check this notebook: https://github.com/pangeo-gallery/osm2020tutorial/blob/master/AWS-notebooks/aws_mur_sst_tutorial_long.ipynb

## From a `netCDF` file

First we download a small sample of data from the CoralTemp product (from NOAA).

In [ ]:
# This part is all ChatGPT:
import xarray as xr
import os
import requests
from datetime import datetime, timedelta

# Base URL components
base_url = "https://www.star.nesdis.noaa.gov/pub/socd/mecb/crw/data/5km/v3.1_op/nc/v1.0/daily/sst/2020/"
file_prefix = "coraltemp_v3.1_"
file_suffix = ".nc"

# Directory to save downloaded files
output_dir = "internal/sst"
os.makedirs(output_dir, exist_ok=True)

# Date range for January 2020
start_date = datetime(2020, 1, 1)
end_date = datetime(2020, 1, 5)

# Loop through each day in January 2020
current_date = start_date
while current_date <= end_date:
    # Format the date as YYYYMMDD
    date_str = current_date.strftime("%Y%m%d")
    # Construct the full URL for the current date's file
    file_url = f"{base_url}{file_prefix}{date_str}{file_suffix}"
    # Path to save the downloaded file
    output_path = os.path.join(output_dir, f"{file_prefix}{date_str}{file_suffix}")
    
    # Download the file
    response = requests.get(file_url)
    if response.status_code == 200:
        with open(output_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {file_url}")
    else:
        print(f"Failed to download: {file_url} (Status code: {response.status_code})")
    
    # Move to the next day
    current_date += timedelta(days=1)

# Directory containing the downloaded NetCDF files
input_dir = output_dir

# List all NetCDF files in the directory
nc_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.nc')]

# Open multiple NetCDF files and concatenate them along the time dimension
combined_ds = xr.open_mfdataset(nc_files, combine='by_coords')

# Save the combined dataset to a new NetCDF file
output_file = "combined_january_2020.nc"
combined_ds.to_netcdf(output_file)

print(f"Combined NetCDF file saved as {output_file}")

This will be an example of how to open a dataset with multiple layers - in that case, only time. But you can have datasets with multiple dimensions! For example depth, different variables, etc.

In [ ]:
# Open the dataset
sst = xr.open_dataset("combined_january_2020.nc")

sst

In this nice overview you can see that there are three coordinates (lat, lon and time) and three variables, being one the analysed_sst. Let's extract the analysed_sst and get a mean of that period.

In [ ]:
sst_analysed = sst["analysed_sst"]

sst_analysed

sst_mean = sst_analysed.mean('time',keep_attrs=True,skipna=False)

sst_mean

In [ ]:
sst_mean.plot()

That is nice, but you can also directly access S3 files and ERDDAP servers. Let's access monthly SST data from this same product:

In [ ]:
coraltemp = xr.open_dataset("https://coastwatch.pfeg.noaa.gov/erddap/griddap/NOAA_DHW_monthly")

coraltemp

Now we see that time is a much bigger set - 481 values. We will subset both the sea_surface_temperature variable and the time coordinate for a smaller period (6 months). Then we will get the average.

In [11]:
ct_subset = coraltemp["sea_surface_temperature"]

In [ ]:
# We also slice the area, to make faster to process
ct_sub_date = ct_subset.sel(time = slice("2020-01-01", "2020-06-01"),
                            latitude = slice(40, 0),
                            longitude = slice(-20, 20))
ct_sub_date

In [ ]:
# This part takes some time, because now the data is being downloaded and processed

ct_mean = ct_sub_date.mean('time', keep_attrs=True,skipna=False)

ct_mean

In [ ]:
ct_mean.plot()